# 📈 Baseline Regression MSE vs Horizon 圖解說明

這段程式碼會要求使用者輸入股票代號（如 `3105`），讀取對應的 `.parquet` 檔案

這張圖呈現的是 **baseline 模型在不同預測 horizon（時間跨度）下的預測誤差（MSE）變化情況**，用於觀察 baseline 模型的穩定性與效能。

---

## 🧠 圖表內容說明：

- **X 軸：Horizon (days)**  
  預測未來幾天的報酬率（例如：3 天後、5 天後、10 天後、20 天後）

- **Y 軸：MSE（Mean Squared Error）**  
  表示 baseline 模型的預測誤差平方平均值。數值越小，代表模型預測越準確。

- **折線圖：不同 baseline 的表現**  
  - `today_return`：使用「今天的報酬率」預測未來報酬率  
  - `zero_return`：預測未來報酬率永遠是 0，作為一種極簡 baseline

---

## 🔍 如何解讀圖形：

1. **MSE 隨 horizon 上升：**
   - 所有 baseline 的 MSE 隨 horizon 增大而線性上升
   - 表示越長期的預測，誤差越大，這是合理且常見的現象

2. **zero_return 明顯優於 today_return：**
   - 幾乎在所有 horizon 下，`zero_return` 的 MSE 都低於 `today_return`
   - 表示「不預測」反而比「用今天的報酬率預測未來」來得更準，這代表市場報酬在短期沒有明顯 auto-correlation

3. **作為 baseline 評估標準：**
   - 若你未來開發的模型 MSE 能明顯低於 `zero_return`，就表示你的模型有學習到有價值的預測訊號
   - 若模型 MSE 甚至高於 `today_return`，就要懷疑是否過擬合或使用了無效特徵

---

## ✅ 使用建議：

- **觀察 MSE 的絕對數值與趨勢變化**
- **將此圖與模型的預測誤差一同繪製比較**
- **可視為模型優化的基準線（Baseline Line）**



In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("../.."))
from config import BASELINE_RESULT_DIR

plt.rcParams.update({'font.size': 12})

stock_code = input("Please enter stock id: ").strip()
filename   = f"{stock_code}.parquet"
input_path = os.path.join(BASELINE_RESULT_DIR, filename)
df = pd.read_parquet(input_path)

mse_summary = {
    col: (df[col].dropna() ** 2).mean()
    for col in df.columns
}
mse_df = pd.DataFrame.from_dict(mse_summary, orient='index', columns=['MSE'])
mse_df = mse_df.sort_index()
# 重新解析 index 成兩欄：horizon + baseline
mse_df = mse_df.reset_index()
mse_df[['horizon', 'baseline']] = mse_df['index'].str.extract(r"(\d+)d_(.+)")
mse_df['horizon'] = pd.to_numeric(mse_df['horizon'], errors='coerce')
mse_df['MSE'] = pd.to_numeric(mse_df['MSE'], errors='coerce')

# 丟掉有缺值的資料
mse_df = mse_df.dropna(subset=['horizon', 'baseline', 'MSE'])

# 製作 pivot 表
pivot_mse = mse_df.pivot_table(index='horizon', columns='baseline', values='MSE', aggfunc='mean')
print(pivot_mse)

pivot_mse.plot(marker='o', figsize=(8, 5))
plt.title(f"Baseline Regression MSE vs Horizon ({stock_id})")
plt.xlabel("Horizon (days)")
plt.ylabel("MSE")
plt.grid(True)
plt.legend(title="Baseline")
plt.tight_layout()
plt.show()
